<a href="https://colab.research.google.com/github/filipediniz/Coursera_Capstone/blob/master/Toronto_week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests_html
!pip install pixiedust
!pip install geocoder

     |████████████████████████████████| 1.2MB 13.2MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13484 sha256=c9497ac75370e9fd02a8c747da8304d462a6e226f88d963dc309f4128a3de6e3
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for pyppeteer: filename=pyppeteer-0.0.25-cp36-none-any.whl size=78360 sha256=b245c2ccdefde024c56674c462f9d4e74ad18a1e6b2457cbbdcc97ee8bed584c
  Stored in directory: /root/.cache/pip/wheels/34/e0/5d/070e22eceecf7ecd5fa4b86bbc18c1c7d0b90e02e9b57f35eb
  Created wheel for parse: filename=parse-1.15.0-cp36-none-any.whl size=23710 sha256=ed5e6c7dcf06b5bf196122b5b57615bdda2b5593199af2956b938a73a1c1f71a
  Stored in directory: /root/.cache/pip/wheels/07/ee/c8/eced0759f09fc635398ab1b8e89c38549b28e5db7fd4a53ba5
Successfully built fake-useragent pyppeteer parse
     |████████████████████████████████| 204

In [2]:
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import numpy as np
import matplotlib.pyplot as mpl
%matplotlib inline
import pixiedust
import requests
from requests_html import HTMLSession
import re as regex

import datetime as dt
from pandas_datareader import data

from IPython.display import clear_output

import pickle

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table USER_PREFERENCES created successfully
Table service_connections created successfully


1. Start by creating a new Notebook for this assignment.

2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 

In [0]:
#%%pixie_debugger

session = HTMLSession()
DF = pd.DataFrame()
ret = session.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')    
try:
    DF = pd.read_html(ret.text, decimal=",", thousands='.', encoding='utf-8')[0]
    #DF.columns = DF.iloc[0]
    #DF = DF.reset_index()
except:
    print('Página sem dados')
    

In [4]:
DF

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


 3. To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [0]:
# Get names of indexes for which column Age has value 30
indexDrop = DF[DF['Borough'] == 'Not assigned'].index
 
# Delete these row indexes from dataFrame
DF.drop(indexDrop , inplace=True)

In [6]:
DF

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
DF[DF['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood


No row has a neighbourhood that is 'Not assigned''

- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [8]:
DF.shape

(210, 3)

4. Submit a link to your Notebook on your Github repository. (10 marks)

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

In [0]:
#import geocoder # import geocoder
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#
#for r in range(0, DF.shape[0]):
#  print('Current row: {}'.format(r))
#  # initialize your variable to None
#  lat_lng_coords = None
#
#  # loop until you get the coordinates
#  g = None
#  while(type(g) == type(None)):    
#    print('Attempting to retrieve coordinates for address {}...'.format('{}, Toronto, Canada'.format(DF.iloc[r]['Postcode'])))
#    geolocator = Nominatim(user_agent="toronto_explorer")
#    g = geolocator.geocode('{}, Toronto, Canada'.format(DF.iloc[r]['Postcode']))      
#    if (type(g) != type(None)):    
#      print('Latitude = {}; longitude = {}'.format(g.latitude, g.longitude))  
#      DF.iloc[r]['Latitude'] = g.latitude
#      DF.iloc[r]['Longitude'] = g.longitude

In [0]:
LL = pd.read_csv('http://cocl.us/Geospatial_data')

In [11]:
for i in range(0, DF.shape[0]):
  for j in range(0, LL.shape[0]):
    if DF.iloc[i]['Postcode'] == LL.iloc[j]['Postal Code']:
      print('Postal code {} coordinates are: latitude = {}; longitude = {}'.format(LL.iloc[j]['Postal Code'], LL.iloc[j]['Latitude'], LL.iloc[j]['Longitude']))
      DF.ix[i, 'Latitude'] = LL.iloc[j]['Latitude']
      DF.ix[i, 'Longitude'] = LL.iloc[j]['Longitude']
      break

Postal code M3A coordinates are: latitude = 43.7532586; longitude = -79.3296565
Postal code M4A coordinates are: latitude = 43.725882299999995; longitude = -79.31557159999998
Postal code M5A coordinates are: latitude = 43.6542599; longitude = -79.3606359
Postal code M6A coordinates are: latitude = 43.718517999999996; longitude = -79.46476329999999
Postal code M6A coordinates are: latitude = 43.718517999999996; longitude = -79.46476329999999
Postal code M7A coordinates are: latitude = 43.6623015; longitude = -79.3894938
Postal code M9A coordinates are: latitude = 43.6678556; longitude = -79.53224240000002
Postal code M1B coordinates are: latitude = 43.806686299999996; longitude = -79.19435340000001
Postal code M1B coordinates are: latitude = 43.806686299999996; longitude = -79.19435340000001
Postal code M3B coordinates are: latitude = 43.745905799999996; longitude = -79.352188
Postal code M4B coordinates are: latitude = 43.7063972; longitude = -79.309937
Postal code M4B coordinates are:

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  


Postal code M5B coordinates are: latitude = 43.6571618; longitude = -79.37893709999999
Postal code M5B coordinates are: latitude = 43.6571618; longitude = -79.37893709999999
Postal code M6B coordinates are: latitude = 43.709577; longitude = -79.44507259999999
Postal code M9B coordinates are: latitude = 43.6509432; longitude = -79.55472440000001
Postal code M9B coordinates are: latitude = 43.6509432; longitude = -79.55472440000001
Postal code M9B coordinates are: latitude = 43.6509432; longitude = -79.55472440000001
Postal code M9B coordinates are: latitude = 43.6509432; longitude = -79.55472440000001
Postal code M9B coordinates are: latitude = 43.6509432; longitude = -79.55472440000001
Postal code M1C coordinates are: latitude = 43.7845351; longitude = -79.16049709999999
Postal code M1C coordinates are: latitude = 43.7845351; longitude = -79.16049709999999
Postal code M1C coordinates are: latitude = 43.7845351; longitude = -79.16049709999999
Postal code M3C coordinates are: latitude = 

In [12]:
DF.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.654260,-79.360636
3,M4A,North York,Victoria Village,43.718518,-79.464763
4,M5A,Downtown Toronto,Harbourfront,43.718518,-79.464763
5,M6A,North York,Lawrence Heights,43.662301,-79.389494
6,M6A,North York,Lawrence Manor,43.667856,-79.532242


Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.